<a href="https://colab.research.google.com/github/KoreanAImaster/Tensorflow/blob/main/%EB%B0%B1%EC%84%9C%EC%9C%A4_%EC%82%B0%EC%B6%9C%EB%AC%BC%EC%B4%88%EC%95%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 모듈 설치
!pip3 install selenium
!pip3 install tqdm
!apt update
!apt install chromium-chromedriver

In [ ]:
# 크롤링 : 데이터 수집
from selenium import webdriver
from bs4 import BeautifulSoup as soups
import os
from tqdm.auto import trange

In [ ]:
def search_selenium(search_name, search_path, search_limit) :
    search_url = "https://www.google.com/search?q=" + str(search_name) + "&hl=ko&tbm=isch"
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920x1080")
    options.add_argument("--disable_gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    browser = webdriver.Chrome(options=options)
    browser.get(search_url)
    os.makedirs(search_path, exist_ok=True)
    image_count = len(browser.find_elements_by_tag_name("img"))
    
    print("로드된 이미지 개수 : ", image_count)

    browser.implicitly_wait(2)
    directory = search_name.replace(" ", "_")
    for i in trange(search_limit) :
        image = browser.find_elements_by_tag_name("img")[i]
        if not directory in os.listdir(search_path):
          os.mkdir(f"{search_path}/{directory}")
        image.screenshot(f"{search_path}/{directory}/{i+1}.png")

    browser.close()

if __name__ == "__main__" :

    search_name = input("검색하고 싶은 키워드 : ")
    search_limit = int(input("원하는 이미지 수집 개수 : "))
    search_path = "/tmp/images"
    # search_maybe(search_name, search_limit, search_path)
    search_selenium(search_name, search_path, search_limit)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#사진들 trainset, testset 분리
import random as r
import shutil
RES_DIR = "/tmp/images"
AI_DIR = "/tmp/RefereeAI"
classes = os.listdir(RES_DIR)
for dirs in classes:
    os.makedirs(f"{AI_DIR}/trainset/{dirs}", exist_ok=True)
    os.makedirs(f"{AI_DIR}/dataset/{dirs}", exist_ok=True)
    data = os.listdir(f"{RES_DIR}/{dirs}")
    trainset = r.sample(data, k=100)
    data = [i for i in data if not (i in trainset)]
    dataset = r.sample(data, k=100)
    for train in trainset:
        shutil.copy(f"{RES_DIR}/{dirs}/{train}", f"{AI_DIR}/trainset/{dirs}")
    for test in dataset:
        shutil.copy(f"{RES_DIR}/{dirs}/{test}", f"{AI_DIR}/dataset/{dirs}")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = f"{AI_DIR}/trainset"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=10,
                                                    class_mode='binary',
                                                    target_size=(200, 200))

VALIDATION_DIR = f"{AI_DIR}/dataset"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=10,
                                                              class_mode='binary',
                                                              target_size=(200, 200))

In [ ]:
# 모델 설계
import tensorflow as tf
from tensorflow.keras.optimizers import Adam


# 컨볼루션 신경망 구축
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='leaky_relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='leaky_relu'),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(train_generator,
                              epochs=10,
                              verbose=1,
                              validation_data=validation_generator)

In [ ]:
# 테스트기 : 사진 업로드 하면 파울인지 아닌지 판단


import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(200, 200))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if np.any(classes[0]>0.5):
    print(fn + " is not foul")
  else:
    print(fn + " is foul")